In [7]:
# Voilà app: Segmentazione con Cellpose e download in memoria
# Permette di caricare un'immagine, segmentarla con Cellpose, calcolare i diametri
# e scaricare un CSV + l'immagine con cerchi sovrapposti senza scrivere su disco.

import io
import base64
from cellpose import models
import numpy as np
import pandas as pd
from skimage.measure import regionprops
from skimage import io as skio
from PIL import Image
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# ------- Helper functions -------

def read_image_from_bytes(b):
    """Legge un'immagine da bytes e la restituisce come numpy array RGB."""
    img = Image.open(io.BytesIO(b)).convert('RGB')
    return np.asarray(img)

def compute_cell_metrics(mask, pixel_size_um=None):
    props = regionprops(mask)
    rows = []
    for i, p in enumerate(props, start=1):
        area = p.area
        diameter_px = 2 * np.sqrt(area / np.pi)
        cy, cx = p.centroid
        minr, minc, maxr, maxc = p.bbox
        row = {
            'cell_id': i,
            'area_px': int(area),
            'diameter_px': float(diameter_px),
            'centroid_y': float(cy),
            'centroid_x': float(cx),
            'bbox_minr': int(minr),
            'bbox_minc': int(minc),
            'bbox_maxr': int(maxr),
            'bbox_maxc': int(maxc),
        }
        if pixel_size_um is not None:
            row['diameter_um'] = float(diameter_px * pixel_size_um)
            row['area_um2'] = float(area * (pixel_size_um ** 2))
        rows.append(row)
    return pd.DataFrame(rows)

def draw_circles_on_image(image, df):
    fig, ax = plt.subplots(figsize=(8, 8))
    if image.ndim == 2:
        ax.imshow(image, cmap='gray')
    else:
        ax.imshow(image)

    for _, row in df.iterrows():
        circ = plt.Circle(
            (row['centroid_x'], row['centroid_y']),
            radius=row['diameter_px'] / 2,
            edgecolor='lime', facecolor='none', lw=1.2
        )
        ax.add_patch(circ)
        ax.text(row['centroid_x'], row['centroid_y'], str(int(row['cell_id'])),
                color='white', fontsize=8, ha='center', va='center')

    ax.axis('off')
    plt.tight_layout()
    return fig

def make_download_link_bytesio(bytes_io, filename, mimetype):
    """Crea un link HTML per scaricare un BytesIO."""
    bytes_io.seek(0)
    b64 = base64.b64encode(bytes_io.read()).decode()
    href = f'<a download="{filename}" href="data:{mimetype};base64,{b64}">{filename}</a>'
    return HTML(href)

# ------- UI Widgets -------

uploader = widgets.FileUpload(accept='image/*', multiple=False)
run_button = widgets.Button(description='Segmenta e genera download', button_style='success')
output = widgets.Output()
scale_input = widgets.FloatText(
    value=0.0,
    description='Pixel size (µm)',
    tooltip='Inserisci la dimensione di un pixel in micrometri. Lascia 0 per disabilitare conversione.'
)
model_type_dd = widgets.Dropdown(options=['cyto', 'nuclei'], value='cyto', description='Model')

# ------- Main callback -------

def on_run_click(b):
    with output:
        clear_output()
        if len(uploader.value) == 0:
            print("Per favore carica prima un'immagine.")
            return

        uploaded_file = uploader.value[0]  # prendi il primo file caricato
        uploaded_filename = uploaded_file['name']   # non 'metadata'
        content = uploaded_file['content']

        try:
            image = read_image_from_bytes(content)
        except Exception as e:
            print("Errore lettura immagine:", e)
            return

        print(f"Immagine caricata: {uploaded_filename}, shape={image.shape}")

        print("Inizializzo il modello Cellpose...")
        try:
            model = models.Cellpose(model_type=model_type_dd.value, gpu=False)
        except Exception as e:
            print("Errore inizializzazione modello:", e)
            return

        print("Eseguo segmentazione (potrebbe impiegare qualche secondo)...")
        try:
            masks, flows, styles, diams = model.eval([image], diameter=None, channels=[0, 0])
        except Exception as e:
            print("Errore durante la segmentazione:", e)
            return

        mask = masks[0]
        print(f"Segmentazione completata. Cellpose diam estimate: {diams}")

        pixel_size = float(scale_input.value) if scale_input.value and scale_input.value > 0 else None
        df = compute_cell_metrics(mask, pixel_size_um=pixel_size)
        n_cells = len(df)
        print(f"Trovate {n_cells} cellule.")

        # CSV in memoria
        csv_buffer = io.BytesIO()
        df.to_csv(csv_buffer, index=False)
        display(HTML("<h4>✅ Analisi completata!</h4>"))
        display(make_download_link_bytesio(csv_buffer, uploaded_filename + '_cell_diameters.csv', 'text/csv'))

        # Immagine con cerchi in memoria
        fig = draw_circles_on_image(image, df)
        img_buffer = io.BytesIO()
        fig.savefig(img_buffer, format='png', dpi=150, bbox_inches='tight')
        plt.close(fig)
        display(make_download_link_bytesio(img_buffer, uploaded_filename + '_with_circles.png', 'image/png'))

        # Anteprima immagine segmentata
        try:
            img_buffer.seek(0)
            preview = skio.imread(img_buffer)
            fig2, ax2 = plt.subplots(figsize=(6,6))
            ax2.imshow(preview)
            ax2.axis('off')
            plt.show()
        except Exception:
            pass

run_button.on_click(on_run_click)

# ------- Layout -------

intro = widgets.HTML(
    "<h3>Voilà: Segmentazione Cellpose → CSV diametri + immagine</h3>"
    "<p>Carica un'immagine di cellule, seleziona il modello, premi <b>Segmenta e genera download</b>.<br>"
    "Otterrai un file CSV con le misure per cellula e un'immagine con cerchi sovrapposti.</p>"
)

controls = widgets.VBox([
    uploader,
    widgets.HBox([model_type_dd, scale_input]),
    run_button
])

app = widgets.VBox([intro, controls, output])
display(app)
